In [1]:
import random
import pandas as pd
import time
import matplotlib.pyplot as plt
import networkx as nx

# Đọc dữ liệu từ file Excel
def doc_du_lieu_tu_excel(duong_dan):
    df = pd.read_excel("/content/drive/MyDrive/TTNT/BTL.xlsx")
    khach_moi = list(set(df['Nguoi1'].tolist() + df['Nguoi2'].tolist()))
    quan_he = {(row['Nguoi1'], row['Nguoi2']): row['MoiQuanHe'] for _, row in df.iterrows()}
    return khach_moi, quan_he

# Điểm quan hệ
diem_quan_he = {
    'anh_chi_em_ruot': 900,
    'cha_me_con': 700,
    'anh_chi_em_ho': 500,
    'ho_hang': 300,
    'ban_be': 100,
    'khong_quen': 0
}

# Giới hạn số người mỗi bàn
gioi_han_so_nguoi = 5
KICH_THUOC_QUAN_THE = 100
SO_THE_HE = 200
TY_LE_DOT_BIEN = 0.1

# Hàm tính điểm thân thiết cho một bàn
def tinh_diem_ban(ban, quan_he):
    diem = 0
    for i in range(len(ban)):
        for j in range(i + 1, len(ban)):
            cap = (ban[i], ban[j]) if (ban[i], ban[j]) in quan_he else (ban[j], ban[i])
            diem += diem_quan_he.get(quan_he.get(cap, 'khong_quen'), 0)
    return diem

# Hàm đánh giá một giải pháp
def danh_gia(giai_phap):
    return sum(tinh_diem_ban(ban, quan_he) for ban in giai_phap)

# Tạo cá thể ngẫu nhiên
def tao_ca_the():
    random.shuffle(khach_moi)
    return [khach_moi[i:i + gioi_han_so_nguoi] for i in range(0, len(khach_moi), gioi_han_so_nguoi)]

# Chọn lọc cá thể tốt nhất
def chon_loc(quan_the):
    return sorted(quan_the, key=danh_gia, reverse=True)[:KICH_THUOC_QUAN_THE // 2]

# Lai ghép hai cá thể
def lai_ghep(ca_the1, ca_the2):
    con = []
    for i in range(min(len(ca_the1), len(ca_the2))):
        if random.random() < 0.5:
            con.append(ca_the1[i])
        else:
            con.append(ca_the2[i])
    return con

# Đột biến cá thể
def dot_bien(ca_the):
    if random.random() < TY_LE_DOT_BIEN:
        ban1, ban2 = random.sample(range(len(ca_the)), 2)
        i, j = random.randint(0, len(ca_the[ban1]) - 1), random.randint(0, len(ca_the[ban2]) - 1)
        ca_the[ban1][i], ca_the[ban2][j] = ca_the[ban2][j], ca_the[ban1][i]
    return ca_the

# Giải thuật di truyền
def thuat_toan_di_truyen():
    quan_the = [tao_ca_the() for _ in range(KICH_THUOC_QUAN_THE)]
    for the_he in range(SO_THE_HE):
        quan_the = chon_loc(quan_the)
        con_cai = [dot_bien(lai_ghep(random.choice(quan_the), random.choice(quan_the))) for _ in range(KICH_THUOC_QUAN_THE)]
        quan_the.extend(con_cai)
        if the_he % 10 == 0:  # Theo dõi tiến trình mỗi 10 thế hệ
            print(f"Thế hệ {the_he}: Điểm tốt nhất = {danh_gia(max(quan_the, key=danh_gia))}")
    giai_phap_tot_nhat = max(quan_the, key=danh_gia)
    return giai_phap_tot_nhat, danh_gia(giai_phap_tot_nhat)

# Vẽ sơ đồ bàn tròn
def ve_so_do_ban_tron(giai_phap):
    for i, ban in enumerate(giai_phap):
        G = nx.Graph()
        G.add_nodes_from(ban)
        for j in range(len(ban)):
            for k in range(j + 1, len(ban)):
                G.add_edge(ban[j], ban[k])
        plt.figure(figsize=(6, 6))
        plt.title(f"Bàn {i + 1}")
        pos = nx.circular_layout(G)
        nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=1000, edge_color='gray')
        plt.show()

# In mối quan hệ giữa các khách trong từng bàn
def in_moi_quan_he(giai_phap, quan_he):
    print("\nChi tiết mối quan hệ trong từng bàn:")
    for i, ban in enumerate(giai_phap):
        print(f"Bàn {i + 1}:")
        for j in range(len(ban)):
            for k in range(j + 1, len(ban)):
                cap = (ban[j], ban[k]) if (ban[j], ban[k]) in quan_he else (ban[k], ban[j])
                moi_quan_he = quan_he.get(cap, 'khong_quen')
                print(f"  {ban[j]} - {ban[k]}: {moi_quan_he}")

# Chạy chương trình
if __name__ == "__main__":
    duong_dan = 'du_lieu.xlsx'  # Đường dẫn file Excel
    khach_moi, quan_he = doc_du_lieu_tu_excel(duong_dan)
    print(f"Số lượng khách mời: {len(khach_moi)}")

    start = time.time()
    giai_phap, diem_tot_nhat = thuat_toan_di_truyen()
    end = time.time()

    print("Sơ đồ chỗ ngồi tối ưu:")
    for i, ban in enumerate(giai_phap):
        print(f"Bàn {i + 1}: {', '.join(ban)}")
    print("Điểm cao nhất:", diem_tot_nhat)
    print(f"Thời gian chạy: {end - start:.2f} giây")

    # In mối quan hệ
    in_moi_quan_he(giai_phap, quan_he)

    # Vẽ sơ đồ bàn tròn
    ve_so_do_ban_tron(giai_phap)


ModuleNotFoundError: No module named 'pandas'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
